<a href="https://www.kaggle.com/code/chongyockeng/ie4483?scriptVersionId=199841082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split

import torchvision
from torchvision import datasets, transforms
import torchvision.models as models
from torchinfo import summary

from timeit import default_timer as timer 
from tqdm.auto import tqdm

import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('/kaggle/input/custom-packages/')
import data_setup, engine, model, utils

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
base_dir = "/kaggle/input/final-project-dataset/datasets"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

print("Train Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Test Directory:", test_dir)

Train Directory: /kaggle/input/final-project-dataset/datasets/train
Validation Directory: /kaggle/input/final-project-dataset/datasets/val
Test Directory: /kaggle/input/final-project-dataset/datasets/test


In [5]:
# Create auto-transforms
transforms = torchvision.models.VGG19_Weights.DEFAULT.transforms() 

In [6]:
# Create DataLoaders
train_dataloader, test_dataloader, val_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    val_dir=val_dir,
    transform=transforms,
    batch_size=32)

In [7]:
# Create vgg model
model_vgg19 = model.create_vgg19(device=device,class_names=class_names).to(device)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 191MB/s]


[INFO] Created new VGG19 model.


In [8]:
# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_vgg19.parameters(),lr=1e-4)

In [9]:
engine.train(model=model_vgg19,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             val_dataloader=val_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=10,
             device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1252 | train_acc: 0.9713 | test_loss: 0.0875 | test_acc: 0.9740
Epoch: 2 | train_loss: 0.0318 | train_acc: 0.9902 | test_loss: 0.0579 | test_acc: 0.9845
Epoch: 3 | train_loss: 0.0209 | train_acc: 0.9944 | test_loss: 0.1056 | test_acc: 0.9812
Epoch: 4 | train_loss: 0.0270 | train_acc: 0.9934 | test_loss: 0.1126 | test_acc: 0.9820
Epoch: 5 | train_loss: 0.0159 | train_acc: 0.9956 | test_loss: 0.1309 | test_acc: 0.9835
Epoch: 6 | train_loss: 0.0193 | train_acc: 0.9959 | test_loss: 0.1607 | test_acc: 0.9805
Epoch: 7 | train_loss: 0.0178 | train_acc: 0.9964 | test_loss: 0.1945 | test_acc: 0.9828
Epoch: 8 | train_loss: 0.0322 | train_acc: 0.9961 | test_loss: 0.1632 | test_acc: 0.9825
Epoch: 9 | train_loss: 0.0199 | train_acc: 0.9968 | test_loss: 0.3469 | test_acc: 0.9802
Epoch: 10 | train_loss: 0.0219 | train_acc: 0.9971 | test_loss: 0.1924 | test_acc: 0.9832


{'train_loss': [0.1252371784289355,
  0.0318129620795379,
  0.020898964154350053,
  0.026979425853890363,
  0.01593957080474527,
  0.01933874359915309,
  0.017755794419270353,
  0.03218169339815317,
  0.01988622404955853,
  0.02185231090863227],
 'train_acc': [0.9713125,
  0.99025,
  0.994375,
  0.993375,
  0.995625,
  0.995875,
  0.9964375,
  0.9960625,
  0.99675,
  0.997125],
 'test_loss': [0.08747697729735228,
  0.057935697317218225,
  0.10561265918349176,
  0.11263420693685433,
  0.1309480045142816,
  0.16074587479795582,
  0.19452435816226327,
  0.16315144040051446,
  0.34685210492204926,
  0.1923863824345748],
 'test_acc': [0.974,
  0.9845,
  0.98125,
  0.982,
  0.9835,
  0.9805,
  0.98275,
  0.9825,
  0.98025,
  0.98325]}

In [10]:
# Save the model
utils.save_model(model=model_vgg19,
                target_dir="models",
                model_name="vgg19_0.pth")

[INFO] Saving model to: models/vgg19_0.pth
